In [61]:
import numpy as np
import cv2
box=np.zeros((3,3,3,3),dtype=int)
pos_map={4:0,1:1,5:2,7:3,3:4,2:5,8:6,6:7,0:8}
face_axis={'r':np.array([1,0,0]),'o':np.array([-1,0,0]),'g':np.array([0,1,0]),'b':np.array([0,-1,0]),'y':np.array([0,0,1]),'w':np.array([0,0,-1])}
secondLayerMiddleEle={'r':np.array([np.array([1,1,0]),np.array([1,0,0])]),'g':np.array([np.array([-1,1,0]),np.array([0,1,0])]),'o':np.array([np.array([-1,-1,0]),np.array([-1,0,0])]),'b':np.array([np.array([1,-1,0]),np.array([0,-1,0])])}
tot_steps=0
steps_dir={}

In [2]:
def mask(image,w,h):
    kernel = np.ones((5,5),np.uint8)
    image = cv2.erode(image,kernel,iterations = 1)
    contour,_=cv2.findContours(image,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    ans=[]
    for cnt in contour:
        area=cv2.contourArea(cnt)
        if(area<1000):
            continue
        m=cv2.moments(cnt)    
        x=m['m10']/m['m00']
        y=m['m01']/m['m00']
        ans.append(min(round((x-w)/(2*w)),2) + (min(round((y-h)/(2*h)),2)*3))
    return ans 

In [3]:
def face_color(image,x,y):
    sq = [None] * 9
    hs=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(hs,np.array([0,70,50]),np.array([10,255,255]))
    mask2=cv2.inRange(hs,np.array([170,20,50]),np.array([180,255,255]))
    maskred=mask1+mask2
    l=mask(maskred,x,y)
    for i in l:
        sq[i]='r'
    maskyel = cv2.inRange(hs,np.array([22,40,40]),np.array([35,255,255]))
    l=mask(maskyel,x,y)
    for i in l:
        sq[i]='y'
    maskora = cv2.inRange(hs,np.array([12,40,40]),np.array([20,255,255]))
    l=mask(maskora,x,y)
    for i in l:
        sq[i]='o'
    maskgre = cv2.inRange(hs,np.array([35,40,40]),np.array([85,255,255]))
    l=mask(maskgre,x,y)
    for i in l:
        sq[i]='g'
    maskblu = cv2.inRange(hs,np.array([90,40,40]),np.array([125,255,255]))
    l=mask(maskblu,x,y)
    for i in l:
        sq[i]='b'
    maskwhi = cv2.inRange(hs,np.array([0,0,215]),np.array([1,1,255]))
    l=mask(maskwhi,x,y)
    for i in l:
        sq[i]='w'
    return sq   

In [4]:
for i in range(6):
    face=cv2.imread("face"+str(i)+".jpeg")
    r = cv2.selectROI(face)
    crop = face[int(r[1]):int(r[1]+r[3]),int(r[0]):int(r[0]+r[2])]
    x1,y1,x2,y2=r
    x=(x2-x1)/6
    y=(y2-y1)/6 
    
    ans=face_color(crop,x,y)
    centre_color=ans[4]
    base=face_axis[ans[4]] 
    #assinging position to faces of squre by a bit of hard coding
    face_sq=[base.copy() for i in range(9)]
    
    if(centre_color in ['r','g','o','b']):
        for i in [0,1,2]:
            face_sq[i][2]=1
        for i in [6,7,8]:
            face_sq[i][2]=-1
    if(centre_color in ['r','y','w']):
        for i in [2,5,8]:
            face_sq[i][1]=1
        for i in [0,3,6]:
            face_sq[i][1]=-1
    if(centre_color in ['o']):
        for i in [2,5,8]:
            face_sq[i][1]=-1
        for i in [0,3,6]:
            face_sq[i][1]=1
    if(centre_color in ['w']):
        for i in [0,1,2]:
            face_sq[i][0]=1
        for i in [6,7,8]:
            face_sq[i][0]=-1
    if(centre_color in ['y']):
        for i in [0,1,2]:
            face_sq[i][0]=-1
        for i in [6,7,8]:
            face_sq[i][0]=1        
    if(centre_color in ['g']):
        for i in [0,3,6]:
            face_sq[i][0]=1
        for i in [2,5,8]:
            face_sq[i][0]=-1 
    if(centre_color in ['b']):
        for i in [0,3,6]:
            face_sq[i][0]=-1
        for i in [2,5,8]:
            face_sq[i][0]=1
    co_to_int={'r':1,'o':2,'g':3,'b':4,'y':5,'w':6,'n':0}
    #ek box matrix me color store kiye hai....a bit like virtal cube
    for i in range(9):
        vec=face_sq[i]
        box[vec[0]+1][vec[1]+1][vec[2]+1]+=co_to_int[ans[i]]*abs(base)
key = cv2.waitKey(0)
if(key == 27):
    cv2.destroyAllWindows()

In [62]:
#Hard coded box matrix
'''box=[[[[6,1,3],
   [6, 4 ,0],
   [2, 4 ,5]],

  [[4 ,0, 5],
   [2, 0 ,0],
   [6, 0, 3]],

  [[2 ,5 ,3],
   [3 ,2 ,0],
   [4 ,1 ,6]]],


 [[[0, 6, 2],
   [0 ,4, 0],
   [0 ,4, 2]],

  [[0, 0 ,6],
   [0 ,0, 0],
   [0 ,0, 5]],

  [[0 ,1, 3],
   [0 ,3 ,0],
   [0 ,5 ,2]]],


 [[[4, 1, 5],
   [1 ,4, 0],
   [5 ,3 ,1]],

  [[1 ,0, 6],
   [1 ,0, 0],
   [5 ,0 ,3]],

  [[6 ,2, 4],
   [5 ,1 ,0],
   [6 ,2 ,3]]]]'''

In [63]:
def face(vec_co,color,face_matrix):
    for i in color:
        x=1
        y=1
        if i in [1,2,3,4]:
            if 5 in color:
                y=0
            if 6 in color:
                y=2
        if i in [1,5,6]:
            if 3 in color:
                x=2
            if 4 in color:
                x=0
        if i in [2]:
            if 4 in color:
                x=2
            if 3 in color:
                x=0                
        if i in [5]:
            if 2 in color:
                y=0
            if 1 in color:
                y=2
        if i in [6]:
            if 1 in color:
                y=0
            if 2 in color:
                y=2
        if i in [3]:
            if 1 in color:
                x=0
            if 2 in color:
                x=2
        if i in [4]:
            if 2 in color:
                x=0
            if 1 in color:
                x=2
        i_color=np.zeros((3,),dtype=int)
        i_color[color.index(i)]=1
        face_matrix[i][pos_map[x+y*3]][0]=np.array(vec_co)
        face_matrix[i][pos_map[x+y*3]][1]=np.array(vec_co)*np.array(i_color)
    return face_matrix    
        

In [64]:
face_matrix=np.zeros((7,9,2,3),dtype=int)
for i in range(3):
    for j in range(3):
        for k in range(3):
            x=i-1
            y=j-1
            z=k-1
            if(x or y or z):
                face_matrix=face([x,y,z],list(box[i][j][k]),face_matrix)       

In [65]:
for i in range(1,7):
    print(face_matrix[i])

[[[ 1  0  0]
  [ 1  0  0]]

 [[ 1  1  0]
  [ 0  1  0]]

 [[ 0  1 -1]
  [ 0  1  0]]

 [[ 1  0 -1]
  [ 1  0  0]]

 [[ 1 -1  0]
  [ 1  0  0]]

 [[ 1 -1  1]
  [ 0  0  1]]

 [[-1 -1 -1]
  [ 0 -1  0]]

 [[-1  1  1]
  [ 0  1  0]]

 [[ 1 -1 -1]
  [ 0 -1  0]]]
[[[-1  0  0]
  [-1  0  0]]

 [[ 0  1  1]
  [ 0  0  1]]

 [[ 0 -1  1]
  [ 0  0  1]]

 [[ 0 -1 -1]
  [ 0  0 -1]]

 [[-1  1  0]
  [ 0  1  0]]

 [[-1 -1  1]
  [-1  0  0]]

 [[ 1  1 -1]
  [ 0  1  0]]

 [[ 1  1  1]
  [ 0  1  0]]

 [[-1  1 -1]
  [-1  0  0]]]
[[[ 0  1  0]
  [ 0  1  0]]

 [[ 1  0  1]
  [ 0  0  1]]

 [[-1  1  0]
  [-1  0  0]]

 [[-1  0  1]
  [ 0  0  1]]

 [[ 0  1 -1]
  [ 0  0 -1]]

 [[-1  1 -1]
  [ 0  0 -1]]

 [[ 1  1  1]
  [ 0  0  1]]

 [[-1 -1 -1]
  [ 0  0 -1]]

 [[ 1 -1  1]
  [ 0 -1  0]]]
[[[ 0 -1  0]
  [ 0 -1  0]]

 [[-1  0 -1]
  [-1  0  0]]

 [[ 1 -1  0]
  [ 0 -1  0]]

 [[-1 -1  0]
  [ 0 -1  0]]

 [[ 0 -1  1]
  [ 0 -1  0]]

 [[ 1 -1 -1]
  [ 1  0  0]]

 [[-1  1  1]
  [-1  0  0]]

 [[ 1  1 -1]
  [ 0  0 -1]]

 [[-1 -1  1]
  [ 0 -

In [66]:
def printallfaces(face_matrix):
    red=np.zeros((3,3),dtype=int)
    orange=np.zeros((3,3),dtype=int)
    green=np.zeros((3,3),dtype=int)
    blue=np.zeros((3,3),dtype=int)
    white=np.zeros((3,3),dtype=int)
    yellow=np.zeros((3,3),dtype=int)
    for i in range(1,7):
        for j in range(9):
            v1=list(face_matrix[i][j][0])
            v2=list(face_matrix[i][j][1])
            if(v2[0]==1):
                x=-v1[2]+1
                y=v1[1]+1
                red[x][y]=i
            elif(v2[0]==-1):
                x=-v1[2]+1
                y=-v1[1]+1
                orange[x][y]=i
            elif(v2[1]==1):
                x=-v1[2]+1
                y=-v1[0]+1
                green[x][y]=i
            elif(v2[1]==-1):
                x=-v1[2]+1
                y=v1[0]+1
                blue[x][y]=i
            elif(v2[2]==1):
                x=v1[0]+1
                y=v1[1]+1
                yellow[x][y]=i
            else:
                x=-v1[0]+1
                y=v1[1]+1
                white[x][y]=i
    print("red",red,sep='\n')
    print("orange",orange,sep='\n')
    print("green",green,sep='\n')
    print("blue",blue,sep='\n')
    print("yellow",yellow,sep='\n')
    print("white",white,sep='\n')
                

In [67]:
printallfaces(face_matrix)

red
[[5 5 6]
 [1 1 5]
 [4 1 6]]
orange
[[4 6 2]
 [3 2 6]
 [2 4 6]]
green
[[2 5 1]
 [1 3 2]
 [2 1 5]]
blue
[[4 4 3]
 [4 4 4]
 [1 6 1]]
yellow
[[5 3 6]
 [2 5 2]
 [1 3 3]]
white
[[5 6 4]
 [2 6 3]
 [3 5 3]]


In [68]:
#direction => center_piece
#clock=1=>clockwise else anticlockwise
def rotate(direction,face_matrix,clock,*data):
    if(not('fake' in data)):
        print("rotating",direction,"in",clock)
    if clock==1:
        areaVec=direction
    else:
        areaVec=-direction
    if(direction[0]!=0):
        pos=0
    elif(direction[1]!=0):
        pos=1
    else:
        pos=2   
    for i in range(1,7):
        for j in range(9):
            if(face_matrix[i][j][0][pos]==direction[pos]):
                face_matrix[i][j][0]=np.cross(face_matrix[i][j][0],areaVec) + direction
                if(face_matrix[i][j][1][pos]!=direction[pos]):
                    face_matrix[i][j][1]=clock*np.cross(face_matrix[i][j][1],direction)

# Random Shuffling

In [69]:
import random
direction=[[1,0,0],[-1,0,0],[0,1,0],[0,-1,0],[0,0,1],[0,0,-1]]
clock=[1,-1]
shuffle=50
for i in range(shuffle):
    rotate(np.array(random.choice(direction)),face_matrix,random.choice(clock))

rotating [0 1 0] in 1
rotating [1 0 0] in 1
rotating [ 0 -1  0] in 1
rotating [ 0  0 -1] in 1
rotating [ 0  0 -1] in -1
rotating [ 0  0 -1] in -1
rotating [-1  0  0] in 1
rotating [-1  0  0] in 1
rotating [0 1 0] in 1
rotating [ 0 -1  0] in -1
rotating [ 0  0 -1] in -1
rotating [1 0 0] in -1
rotating [0 1 0] in 1
rotating [ 0  0 -1] in -1
rotating [-1  0  0] in 1
rotating [0 1 0] in -1
rotating [0 0 1] in 1
rotating [0 0 1] in -1
rotating [-1  0  0] in 1
rotating [0 0 1] in -1
rotating [ 0 -1  0] in -1
rotating [-1  0  0] in -1
rotating [0 1 0] in 1
rotating [ 0 -1  0] in 1
rotating [1 0 0] in 1
rotating [ 0  0 -1] in 1
rotating [1 0 0] in 1
rotating [0 1 0] in 1
rotating [ 0  0 -1] in -1
rotating [-1  0  0] in -1
rotating [ 0  0 -1] in -1
rotating [ 0  0 -1] in -1
rotating [0 1 0] in -1
rotating [ 0 -1  0] in -1
rotating [ 0 -1  0] in 1
rotating [ 0  0 -1] in 1
rotating [0 0 1] in -1
rotating [0 0 1] in 1
rotating [ 0 -1  0] in -1
rotating [ 0 -1  0] in 1
rotating [-1  0  0] in 1
rota

## Step 1

In [70]:
def faltu(x,y):
    if(x[0]==1):
        if(y[1]==1):
            move=1
        else:
            move=-1
    elif(x[0]==-1):
        if(y[1]==1):
            move=-1
        else:
            move=1
    elif(x[1]==1):
        if(y[0]==1):
            move=-1
        else:
            move=1
    else:
        if(y[0]==1):
            move=1
        else:
            move=-1
    return move        

In [71]:
def step1(face,face_matrix):
    face=np.array(face)
    if(face[0]==1):
        sq=1
    elif(face[0]==-1):
        sq=3
    elif(face[1]==1):
        sq=2
    else:
        sq=4
    print("step0")
    steps=0
    if(face_matrix[6][sq][0][2]==1):
        if(face_matrix[6][sq][1][2]!=1):
            if((face_matrix[6][sq][1][0]==face[0])and(face_matrix[6][sq][1][1]==face[1])):
                rotate(face,face_matrix,1)
                steps+=1
            elif((face_matrix[6][sq][1][0]==-face[0])and(face_matrix[6][sq][1][1]==-face[1])):
                dir1=(face_matrix[6][sq][1]).copy()
                rotate(dir1,face_matrix,-1)
                rotate(np.array([0,0,-1]),face_matrix,1)
                dir2=np.bitwise_xor(face_matrix[6][sq][0],face_matrix[6][sq][1])
                rotate(dir2,face_matrix,1)
                rotate(np.array([0,0,-1]),face_matrix,-1)
                rotate(dir1,face_matrix,1)
                steps+=5
                return steps
            else:
                move2=faltu(face_matrix[6][sq][1],face)
                dir1=(face_matrix[6][sq][1]).copy()
                rotate(dir1,face_matrix,move2)
                rotate(face,face_matrix,-move2)
                rotate(dir1,face_matrix,-move2)
                steps+=3
                return steps
    print("step1")
    if(face_matrix[6][sq][0][2]==-1):
        if(face_matrix[6][sq][1][2]==-1):
            if((face_matrix[6][sq][0][0]==face[0]) and (face_matrix[6][sq][0][1]==face[1])):
                return steps
            else:
                temp=np.bitwise_xor(face_matrix[6][sq][0],face_matrix[6][sq][1])
                if((temp[0]==-face[0])and(temp[1]==-face[1])):
                    rotate(temp,face_matrix,1)
                    rotate(temp,face_matrix,1)
                    steps+=2
                else:
                    move2=faltu(face,temp)
                    rotate(temp,face_matrix,move2)
                    rotate(np.array([0,0,-1]),face_matrix,move2)
                    rotate(temp,face_matrix,-move2)
                    rotate(np.array([0,0,-1]),face_matrix,-move2)
                    steps+=4
                    return steps      
        else:
            if((face_matrix[6][sq][0][0]==-face[0]) and (face_matrix[6][sq][0][1]==-face[1])):
                rotate(face_matrix[6][sq][1],face_matrix,1)
                rotate(np.array([0,0,-1]),face_matrix,1)
                rotate(np.bitwise_xor(face_matrix[6][sq][0],face_matrix[6][sq][1]),face_matrix,1)
                rotate(np.array([0,0,-1]),face_matrix,-1)
                steps+=4
                return steps
            elif((face_matrix[6][sq][0][0]==face[0]) and (face_matrix[6][sq][0][1]==face[1])):
                rotate(face_matrix[6][sq][1],face_matrix,1)
                rotate(np.array([0,0,-1]),face_matrix,-1)
                rotate(np.bitwise_xor(face_matrix[6][sq][0],face_matrix[6][sq][1]),face_matrix,1)
                rotate(np.array([0,0,-1]),face_matrix,1)
                steps+=4
                return steps
            else:
                move2=faltu(face,face_matrix[6][sq][0])
                rotate(face_matrix[6][sq][1],face_matrix,move2)  
                steps+=1
    print("step2")
    if(face_matrix[6][sq][0][2]==0):
        temp=np.bitwise_xor(face_matrix[6][sq][0],face_matrix[6][sq][1])
        if(list(temp)==list(face)):
            move2=faltu(face_matrix[6][sq][1],face_matrix[6][sq][0])
            rotate(face,face_matrix,-move2)
            steps+=1
            return steps
        elif(list(face_matrix[6][sq][1])==list(face)):
            move2=faltu(face_matrix[6][sq][1],temp)
            rotate(np.array([0,0,-1]),face_matrix,move2)
            rotate(temp,face_matrix,-move2)
            rotate(np.array([0,0,-1]),face_matrix,-move2)
            steps+=3
            return steps
        elif((face_matrix[6][sq][1][0]==-face[0])and(face_matrix[6][sq][1][1]==-face[1])):
            move2=faltu(face_matrix[6][sq][1],temp)
            rotate(np.array([0,0,-1]),face_matrix,-move2)
            rotate(temp,face_matrix,-move2)
            rotate(np.array([0,0,-1]),face_matrix,move2)
            steps+=3
            return steps
        else:
            temp2=face_matrix[6][sq][1].copy()
            rotate(temp2,face_matrix,1)
            rotate(temp2,face_matrix,1)
            move2=faltu(face_matrix[6][sq][1],face_matrix[6][sq][0])
            rotate(face,face_matrix,-move2)
            rotate(temp2,face_matrix,1)
            rotate(temp2,face_matrix,1)
            steps+=5
            return steps
    print("step3")
    if(face_matrix[6][sq][0][2]==1):
        if((face_matrix[6][sq][0][0]==face[0])and(face_matrix[6][sq][0][1]==face[1])):
            pass
        elif((face_matrix[6][sq][0][0]==-face[0])and(face_matrix[6][sq][0][1]==-face[1])):
            rotate(np.array([0,0,1]),face_matrix,1)
            rotate(np.array([0,0,1]),face_matrix,1)
            steps+=2
        else:
            move2=faltu(np.bitwise_xor(face_matrix[6][sq][0],face_matrix[6][sq][1]),face)
            rotate(np.array([0,0,1]),face_matrix,-move2)
            steps+=1      
        rotate(face,face_matrix,1)
        rotate(face,face_matrix,1)
        steps+=2
    return steps

In [72]:
def optimizestep1(face_matrix):
    itr=[[1,0,0], [0,1,0], [-1,0,0],[0,-1,0]]
    steps=1000
    for i in itr:
        now1=0
        face_matrix1=face_matrix.copy()
        now1+=step1(i,face_matrix1)
        for j in itr:
            if j not in [i]:
                now2=now1
                face_matrix2=face_matrix1.copy()
                now2+=step1(j,face_matrix2)
                for k in itr:
                    if k not in [i,j]:
                        now3=now2
                        face_matrix3=face_matrix2.copy()
                        now3+=step1(k,face_matrix3)
                        for l in itr:
                            if l not in [i,j,k]:
                                now4=now3
                                now4+=step1(l,face_matrix3)
                                if(now4<steps):
                                    steps=now4
                                    ans=[i,j,k,l]
    return ans                                

In [73]:
ans=optimizestep1(face_matrix)

step0
step1
step2
step3
rotating [0 0 1] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in 1
rotating [1 0 0] in 1
step0
step1
rotating [0 1 0] in 1
rotating [ 0  0 -1] in -1
rotating [1 0 0] in 1
rotating [ 0  0 -1] in 1
step0
step1
step2
rotating [0 1 0] in 1
rotating [0 1 0] in 1
rotating [-1  0  0] in -1
rotating [0 1 0] in 1
rotating [0 1 0] in 1
step0
step1
step2
rotating [ 0 -1  0] in -1
step0
step1
rotating [-1  0  0] in -1
step2
rotating [ 0 -1  0] in -1
step0
step1
step2
rotating [0 1 0] in 1
rotating [0 1 0] in 1
rotating [-1  0  0] in -1
rotating [0 1 0] in 1
rotating [0 1 0] in 1
step0
step1
step2
step3
rotating [0 0 1] in 1
rotating [0 0 1] in 1
rotating [-1  0  0] in 1
rotating [-1  0  0] in 1
step0
step1
rotating [0 1 0] in 1
rotating [ 0  0 -1] in -1
rotating [1 0 0] in 1
rotating [ 0  0 -1] in 1
step0
rotating [-1  0  0] in 1
rotating [ 0 -1  0] in -1
rotating [-1  0  0] in -1
step0
rotating [-1  0  0] in 1
rotating [ 0 -1  0] in -1
rotating [-1  0  0] in -1
step0
step1


In [74]:
ans

[[0, -1, 0], [0, 1, 0], [1, 0, 0], [-1, 0, 0]]

In [75]:
steps=0
for i in range(4):
    print("\nsetting ",i+1," piece")
    steps+=step1(ans[i],face_matrix)


setting  1  piece
step0
step1
rotating [-1  0  0] in -1
step2
rotating [ 0 -1  0] in -1

setting  2  piece
step0
step1
rotating [0 1 0] in 1
rotating [ 0  0 -1] in -1
rotating [1 0 0] in 1
rotating [ 0  0 -1] in 1

setting  3  piece
step0
step1
step2
rotating [ 0  0 -1] in -1
rotating [ 0 -1  0] in 1
rotating [ 0  0 -1] in 1

setting  4  piece
step0
step1
step2
rotating [ 0  0 -1] in 1
rotating [ 0 -1  0] in -1
rotating [ 0  0 -1] in -1


In [76]:
print("total steps taken:- ",steps)
printallfaces(face_matrix)
tot_steps+=steps
steps_dir["White cross"]=steps

total steps taken:-  12
red
[[6 1 2]
 [3 1 2]
 [2 1 4]]
orange
[[6 4 1]
 [1 2 5]
 [5 2 2]]
green
[[6 2 1]
 [5 3 4]
 [2 3 1]]
blue
[[5 1 4]
 [3 4 2]
 [3 4 3]]
yellow
[[4 5 3]
 [5 5 4]
 [1 3 4]]
white
[[6 6 5]
 [6 6 6]
 [5 6 3]]


In [77]:
def optimisingRotaion(face_matrix,p):
    steps=[0,0]
    a=[-1,1]
    for i in a:
        face_matrix3=face_matrix.copy()
        if p==5:
            while sum(face_matrix3[6][p][0]-np.array([0,0,1]))!=2:
                rotate(np.array([0,0,1]),face_matrix3,i,"fake")
                
                steps[int((i+1)/2)]+=1
        elif p==6:
            while np.array_equal(face_matrix3[6][p][0],np.array([-1,1,1]))!=1:
                rotate(np.array([0,0,1]),face_matrix3,i,"fake")
                
                steps[int((i+1)/2)]+=1
        elif p==8:
            while np.array_equal(face_matrix3[6][p][0],np.array([1,-1,1]))!=1:
                rotate(np.array([0,0,1]),face_matrix3,i,"fake")
                
                steps[int((i+1)/2)]+=1
        elif p==7:
            while sum(face_matrix3[6][p][0]-np.array([0,0,1]))!=-2:
                rotate(np.array([0,0,1]),face_matrix3,i,"fake")
                
                steps[int((i+1)/2)]+=1
    return a[steps.index(min(steps))] 

In [78]:
def specificRotations(face_matrix,face,j):
    rotate(face,face_matrix,j)
    rotate(np.array([0,0,1]),face_matrix,j)
    rotate(face,face_matrix,-j)

In [79]:
wCorners=[5,6,7,8]
def initialRotation(face_matrix,p,nor):
    if p==5:
        pk=optimisingRotaion(face_matrix,p)
        while sum(face_matrix[6][p][0]-np.array([0,0,1]))!=2:
            rotate(np.array([0,0,1]),face_matrix,pk)
            nor[0]+=1
    elif p==6:
        pk=optimisingRotaion(face_matrix,p)
        while np.array_equal(face_matrix[6][p][0],np.array([-1,1,1]))!=1:
            rotate(np.array([0,0,1]),face_matrix,pk)
            nor[0]+=1
    elif p==8:
        pk=optimisingRotaion(face_matrix,p)
        while np.array_equal(face_matrix[6][p][0],np.array([1,-1,1]))!=1:
            rotate(np.array([0,0,1]),face_matrix,pk)
            nor[0]+=1
    elif p==7:
        pk=optimisingRotaion(face_matrix,p)
        while sum(face_matrix[6][p][0]-np.array([0,0,1]))!=-2:
            rotate(np.array([0,0,1]),face_matrix,pk)
            nor[0]+=1
 
    
def settingCorners(face_matrix,p,nor):
    b=0
    map1={5:np.array([1,1,-1]),6:np.array([-1,1,-1]),7:np.array([-1,-1,-1]),8:np.array([1,-1,-1])}
    map2={1:np.array([1,0,0]),2:np.array([-1,0,0]),3:np.array([0,1,0]),4:np.array([0,-1,0])}
    map3={5:np.array([1,0,0]),6:np.array([0,1,0]),7:np.array([-1,0,0]),8:np.array([0,-1,0])}
    if(face_matrix[6][p][0][2]==-1):
        print(p,'S3')
        if(np.array_equal(face_matrix[6][p][0],map1[p])==1):
            if(np.array_equal(face_matrix[6][p][1],np.array([0,0,-1]))!=1):
                ux=face_matrix[6][p][0]-face_matrix[6][p][1]-np.array([0,0,-1])
                vx=np.cross(face_matrix[6][p][1],ux)
                copy=face_matrix[6][p][1].copy()
                if(sum(vx)<0):
                    specificRotations(face_matrix,copy,1)
                    nor[0]+=3
                else:
                    specificRotations(face_matrix,copy,-1)
                    nor[0]+=3 
        else:
            t=0
            wc=wCorners.copy() 
            wc.remove(p)
            for i in wc:
                   if(np.array_equal(face_matrix[6][p][0],map1[i])==1):
                        bbl=face_matrix[6][p][1].copy()
                        t=1
                        print(bbl)
                        break
            if(t==1):            
                if(np.array_equal(bbl,np.array([0,0,-1]))==1):
                    specificRotations(face_matrix,map3[p],-1)
                    nor[0]+=3
                else:
                    v1=face_matrix[6][p][0]-bbl-np.array([0,0,-1])
                    v2=np.cross(bbl,v1)
                    if(sum(v2)<0):
                        specificRotations(face_matrix,bbl,1)
                        nor[0]+=3
                    else:
                        specificRotations(face_matrix,bbl,-1)
                        nor[0]+=3            
    z=[]
    if face_matrix[6][p][0][2]==1 and face_matrix[6][p][1][2]==1:
        print(p,'S2')
        initialRotation(face_matrix,p,nor) 
        for i in range(3):
            e=[]
            for j in range(3):
                if(i==j):
                    e.append(face_matrix[6][p][0][i])
                else:
                    e.append(0)
            if(e!=list(face_matrix[6][p][1])):
                z.append(np.array(e))             
        v5=np.cross(z[0],z[1])
        if(sum(v5)>0):
            rotate(z[0],face_matrix,-1)
            rotate(np.array([0,0,1]),face_matrix,1)
            rotate(np.array([0,0,1]),face_matrix,1)
            rotate(z[0],face_matrix,1)
            nor[0]+=4
        else:
            rotate(z[0],face_matrix,1)
            rotate(np.array([0,0,1]),face_matrix,1)
            rotate(np.array([0,0,1]),face_matrix,1)
            rotate(z[0],face_matrix,-1)
            nor[0]+=4
    k=np.array([0,0,0])
    if face_matrix[6][p][0][2]==1 and face_matrix[6][p][1][2]!=1 and face_matrix[6][p][1][2]!=-1:
        print(p,"S1")
        initialRotation(face_matrix,p,nor)
        print(face_matrix[6][p][1])
        k=face_matrix[6][p][1].copy()          
        v3=face_matrix[6][p][0]-k-np.array([0,0,1])
        v4=np.cross(k,v3)
        if(sum(v4)<0):   
            specificRotations(face_matrix,k,1)
            nor[0]+=3
        if(sum(v4)>0): 
            specificRotations(face_matrix,k,-1)
            nor[0]+=3


    

In [80]:
import itertools
def optimisingSolvingWhiteCorners(face_matrix):
    combinations=list(itertools.permutations(wCorners))
    print(combinations)
    cmp=[]
    for i in combinations:
        nor=[0]
        kg=face_matrix.copy()
        for m in range(3):
            for j in i:
                settingCorners(kg,j,nor)    
        cmp.append(nor) 
    print(cmp)
    return combinations[cmp.index(min(cmp))]
    

In [81]:

ans=optimisingSolvingWhiteCorners(face_matrix)
print(ans)


[(5, 6, 7, 8), (5, 6, 8, 7), (5, 7, 6, 8), (5, 7, 8, 6), (5, 8, 6, 7), (5, 8, 7, 6), (6, 5, 7, 8), (6, 5, 8, 7), (6, 7, 5, 8), (6, 7, 8, 5), (6, 8, 5, 7), (6, 8, 7, 5), (7, 5, 6, 8), (7, 5, 8, 6), (7, 6, 5, 8), (7, 6, 8, 5), (7, 8, 5, 6), (7, 8, 6, 5), (8, 5, 6, 7), (8, 5, 7, 6), (8, 6, 5, 7), (8, 6, 7, 5), (8, 7, 5, 6), (8, 7, 6, 5)]
5 S1
rotating [0 0 1] in 1
[0 1 0]
rotating [0 1 0] in 1
rotating [0 0 1] in 1
rotating [0 1 0] in -1
6 S3
[ 0  0 -1]
rotating [0 1 0] in -1
rotating [0 0 1] in -1
rotating [0 1 0] in 1
7 S1
rotating [0 0 1] in 1
[ 0 -1  0]
rotating [ 0 -1  0] in 1
rotating [0 0 1] in 1
rotating [ 0 -1  0] in -1
8 S2
rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in -1
8 S1
rotating [0 0 1] in -1
[1 0 0]
rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in -1
5 S3
6 S1
[0 1 0]
rotating [0 1 0] in -1
rotating [0 0 1] in -1
rotating [0 1 0] in 1
7 S3
8 S3
5 S3
6 S3
7 S3
8 S3
5 S1
rotating [0 0 1] in 1
[0 1 0]
rotating [0 1 0] i

rotating [-1  0  0] in -1
8 S1
rotating [0 0 1] in 1
[1 0 0]
rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in -1
7 S3
5 S3
6 S1
rotating [0 0 1] in 1
[0 1 0]
rotating [0 1 0] in -1
rotating [0 0 1] in -1
rotating [0 1 0] in 1
8 S3
7 S3
5 S3
6 S3
8 S3
7 S1
rotating [0 0 1] in -1
rotating [0 0 1] in -1
[ 0 -1  0]
rotating [ 0 -1  0] in 1
rotating [0 0 1] in 1
rotating [ 0 -1  0] in -1
5 S1
rotating [0 0 1] in -1
[0 1 0]
rotating [0 1 0] in 1
rotating [0 0 1] in 1
rotating [0 1 0] in -1
8 S1
rotating [0 0 1] in -1
rotating [0 0 1] in -1
[1 0 0]
rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in -1
6 S1
rotating [0 0 1] in 1
[0 1 0]
rotating [0 1 0] in -1
rotating [0 0 1] in -1
rotating [0 1 0] in 1
7 S3
5 S3
8 S3
6 S3
7 S3
5 S3
8 S3
6 S3
7 S1
rotating [0 0 1] in -1
rotating [0 0 1] in -1
[ 0 -1  0]
rotating [ 0 -1  0] in 1
rotating [0 0 1] in 1
rotating [ 0 -1  0] in -1
6 S3
[ 0  0 -1]
rotating [0 1 0] in -1
rotating [0 0 1] in -1
rotating [0 1 0] in 1
5 S1
rot

In [82]:
def solvingWhiteCorners(face_matrix):
    nor=[0]
    for j in range(3):
        for i in ans:
            settingCorners(face_matrix,i,nor)
    print("steps"+str(nor)) 
    return nor[0]
step=solvingWhiteCorners(face_matrix) 
tot_steps+=step
steps_dir["White corner"]=step

5 S1
rotating [0 0 1] in 1
[0 1 0]
rotating [0 1 0] in 1
rotating [0 0 1] in 1
rotating [0 1 0] in -1
7 S1
[ 0 -1  0]
rotating [ 0 -1  0] in 1
rotating [0 0 1] in 1
rotating [ 0 -1  0] in -1
8 S1
rotating [0 0 1] in -1
[ 0 -1  0]
rotating [ 0 -1  0] in -1
rotating [0 0 1] in -1
rotating [ 0 -1  0] in 1
6 S1
rotating [0 0 1] in -1
[-1  0  0]
rotating [-1  0  0] in 1
rotating [0 0 1] in 1
rotating [-1  0  0] in -1
5 S3
7 S3
8 S3
6 S3
5 S3
7 S3
8 S3
6 S3
steps[15]


In [83]:
printallfaces(face_matrix)

red
[[4 2 3]
 [3 1 5]
 [1 1 1]]
orange
[[5 2 5]
 [4 2 2]
 [2 2 2]]
green
[[5 1 2]
 [1 3 5]
 [3 3 3]]
blue
[[2 3 5]
 [5 4 1]
 [4 4 4]]
yellow
[[4 4 3]
 [5 5 4]
 [1 3 1]]
white
[[6 6 6]
 [6 6 6]
 [6 6 6]]


# Second Layer


Below we have tried to solve the second layer of the cube

In [84]:
def ansIsSame(ans1,ans2):
    if list(ans1[0])==list(ans2[0]) and ans1[1]==ans2[1]:
        return True
    else:
        return False

In [85]:
def secondLayerEle(face_matrix):
    elem_req=[]
    index=[]
    red_green=[face_matrix[1,2],face_matrix[3,4]]
    green_orange=[face_matrix[3,2],face_matrix[2,4]]
    orange_blue=[face_matrix[2,2],face_matrix[4,4]]
    blue_red=[face_matrix[4,2],face_matrix[1,4]]
    
    order=['rg','go','ob','br']
    
    elem_req.append(red_green)
    elem_req.append(green_orange)
    elem_req.append(orange_blue)
    elem_req.append(blue_red)
    
    index.append([[1,2],[3,4]])
    index.append([[3,2],[2,4]])
    index.append([[2,2],[4,4]])
    index.append([[4,2],[1,4]])
    
    return elem_req,order,index

In [86]:
def rightTriger(face,face_matrix):
    ans=[]
    right_face=np.cross(np.array([0,0,1]),face)
    
    rotate(right_face,face_matrix,1)
    ans.append((right_face,1))
    
    rotate(np.array([0,0,1]),face_matrix,1)
    ans.append(([0,0,1],1))
    
    rotate(right_face,face_matrix,-1)
    ans.append((right_face,-1))
   
    return ans

In [87]:
def leftTriger(face,face_matrix):
    left_face=np.cross(face,np.array([0,0,1]))
    ans=[]
    
    rotate(left_face,face_matrix,-1)
    ans.append((left_face,-1))
    
    rotate(np.array([0,0,1]),face_matrix,-1)
    ans.append(([0,0,1],-1))
    
    rotate(left_face,face_matrix,1)
    ans.append((left_face,1))
    
    return ans

In [88]:
def firstCaseSecondLayer(face_matrix,side_ele,side_color,top_ele,top_color):
    final_dir=face_axis[side_color]
    curr_dir=np.copy(side_ele[1])
    ans=[]
    while(list(curr_dir)!=list(final_dir)):
        rotate(np.array([0,0,1]),face_matrix,1)
        curr_dir=np.cross(curr_dir,np.array([0,0,1]))
        ans.append(([0,0,1],1))
    #Now the side color matches the center piece
    top_color_dir=face_axis[top_color]
    cross_prd=np.cross(final_dir,top_color_dir)
    if list(cross_prd)==[0,0,1]:
        # the top color centre_piece is in right
        rotate(np.array([0,0,1]),face_matrix,1)
        ans.append(([0,0,1],1))
        ans=ans+rightTriger(final_dir,face_matrix)
        # Now a white piece is removed so I have to get it back to the same 
                
        rotate(np.array([0,0,1]),face_matrix,-1)
        ans.append(([0,0,1],-1))
        ans=ans+leftTriger(top_color_dir,face_matrix)
        # Now its done
    else:
        #the top color center_piece is in left
        rotate(np.array([0,0,1]),face_matrix,-1)
        ans.append(([0,0,1],-1))
        ans=ans+leftTriger(final_dir,face_matrix)
        # Now a white piece is removed so I have to get it back to the same 
                
        rotate(np.array([0,0,1]),face_matrix,1)
        ans.append(([0,0,1],1))
        ans=ans+rightTriger(top_color_dir,face_matrix)
    return ans

In [89]:
def solveSecondLayer(face_matrix,ele,order,index):
    ans=[]
    for i in range(len(ele)):
        fir_cubit,sec_cubit=ele[i]
        fir_color,sec_color=order[i]
        fir_index,sec_index=index[i]
        if list(secondLayerMiddleEle[fir_color][0])==list(fir_cubit[0]) and list(secondLayerMiddleEle[fir_color][1])==list(fir_cubit[1]):
            continue
        if list(fir_cubit[1])!=[0,0,1] and list(sec_cubit[1])!=[0,0,1]:
            cross_prd=np.cross(fir_cubit[1],sec_cubit[1])
            if cross_prd[2]==1:
                #the second cubit is in left
                ans=ans+leftTriger(sec_cubit[1],face_matrix)
                rotate(np.array([0,0,1]),face_matrix,1)
                ans.append([[0,0,1],1])
                ans=ans+rightTriger(fir_cubit[1],face_matrix)
                
            else:
                #the first cubit is in left
                ans=ans+leftTriger(fir_cubit[1],face_matrix)
                rotate(np.array([0,0,1]),face_matrix,1)
                ans.append([[0,0,1],1])
                ans=ans+rightTriger(sec_cubit[1],face_matrix)
                
            fir_cubit=face_matrix[fir_index[0],fir_index[1]]
            sec_cubit=face_matrix[sec_index[0],sec_index[1]]
        if list(fir_cubit[1]) == [0,0,1]:
            ans=ans+firstCaseSecondLayer(face_matrix,sec_cubit,sec_color,fir_cubit,fir_color)
        elif list(sec_cubit[1])==[0,0,1]:
            ans=ans+firstCaseSecondLayer(face_matrix,fir_cubit,fir_color,sec_cubit,sec_color)
            
    return ans

In [90]:
def optimiseAnsList(ans):
    optimisedAns=[]
    length=len(ans)
    i=0
    while(i<length):
        if (len(optimisedAns)>=2 and list(optimisedAns[-1][0])==list(optimisedAns[-2][0]) and optimisedAns[-1][1]==-optimisedAns[-2][1]):
            del optimisedAns[-1]
            del optimisedAns[-1]
        if (i<=length-3 and ansIsSame(ans[i],ans[i+1]) and ansIsSame(ans[i+1],ans[i+2])):
            optimisedAns.append([ans[i][0],-ans[i][1]])
            i+=3
        elif (len(optimisedAns)!=0 and list(optimisedAns[-1][0])==list(ans[i][0]) and optimisedAns[-1][1]==-ans[i][1]):
            del optimisedAns[-1]
            i+=1
        else:
            optimisedAns.append(ans[i])
            i+=1
    return optimisedAns    

In [91]:
def optimisedSolveSecondLayer(face_matrix):
    ele,order,index=secondLayerEle(face_matrix)
    perm_ele=list(itertools.permutations(ele))
    perm_order=list(itertools.permutations(order))
    perm_index=list(itertools.permutations(index))
    
    superOptimisedAns=[]
    
    min_ele=[]
    min_order=[]
    min_index=[]
    
    
    
    for i in range(len(perm_ele)):
        face_matrix_copy=face_matrix.copy()
        ele_i=perm_ele[i]
        order_i=perm_order[i]
        index_i=perm_index[i]
        
        ans=solveSecondLayer(face_matrix_copy,ele_i,order_i,index_i)
        
        optimisedAns=optimiseAnsList(ans)
        
        if i==0:
            superOptimisedAns=optimisedAns
            min_ele=ele_i
            min_order=order_i
            min_index=index_i
            
        elif len(superOptimisedAns)>len(optimisedAns):
            superOptimisedAns=optimisedAns
            min_ele=ele_i
            min_order=order_i
            min_index=index_i
        print("Got",i)
    
    _=solveSecondLayer(face_matrix,min_ele,min_order,min_index)
    return superOptimisedAns

In [92]:
optimisedAns=optimisedSolveSecondLayer(face_matrix)

rotating [ 0 -1  0] in -1
rotating [0 0 1] in -1
rotating [ 0 -1  0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in -1
rotating [0 0 1] in 1
rotating [0 0 1] in 1
rotating [0 0 1] in 1
rotating [0 1 0] in 1
rotating [0 0 1] in 1
rotating [0 1 0] in -1
rotating [0 0 1] in -1
rotating [1 0 0] in -1
rotating [0 0 1] in -1
rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotating [0 0 1] in 1
rotating [0 0 1] in -1
rotating [0 1 0] in -1
rotating [0 0 1] in -1
rotating [0 1 0] in 1
rotating [0 0 1] in 1
rotating [-1  0  0] in 1
rotating [0 0 1] in 1
rotating [-1  0  0] in -1
rotating [0 0 1] in 1
rotating [ 0 -1  0] in 1
rotating [0 0 1] in 1
rotating [ 0 -1  0] in -1
rotating [0 0 1] in -1
rotating [-1  0  0] in -1
rotating [0 0 1] in -1
rotating [-1  0  0] in 1
rotating [0 0 1] in 1
rotating [0 0 1] in -1
rotating [ 0 -1  0] in -1
rotating [0 0 1] in -1
rotating [ 0 -1  0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotati

rotating [0 0 1] in -1
rotating [ 0 -1  0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in -1
Got 8
rotating [0 0 1] in 1
rotating [0 0 1] in 1
rotating [0 0 1] in -1
rotating [0 1 0] in -1
rotating [0 0 1] in -1
rotating [0 1 0] in 1
rotating [0 0 1] in 1
rotating [-1  0  0] in 1
rotating [0 0 1] in 1
rotating [-1  0  0] in -1
rotating [0 0 1] in 1
rotating [ 0 -1  0] in 1
rotating [0 0 1] in 1
rotating [ 0 -1  0] in -1
rotating [0 0 1] in -1
rotating [-1  0  0] in -1
rotating [0 0 1] in -1
rotating [-1  0  0] in 1
rotating [0 0 1] in 1
rotating [0 0 1] in -1
rotating [ 0 -1  0] in -1
rotating [0 0 1] in -1
rotating [ 0 -1  0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in -1
rotating [ 0 -1  0] in -1
rotating [0 0 1] in -1
rotating [ 0 -1  0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in -1
rotating [0 0 1] in 1
rotating [0 0 1] in 1
rotating [0 0 1] in 

rotating [0 0 1] in -1
rotating [0 1 0] in 1
rotating [0 0 1] in 1
rotating [-1  0  0] in 1
rotating [0 0 1] in 1
rotating [-1  0  0] in -1
rotating [ 0 -1  0] in -1
rotating [0 0 1] in -1
rotating [ 0 -1  0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in -1
rotating [0 0 1] in 1
rotating [0 0 1] in 1
rotating [0 1 0] in 1
rotating [0 0 1] in 1
rotating [0 1 0] in -1
rotating [0 0 1] in -1
rotating [1 0 0] in -1
rotating [0 0 1] in -1
rotating [1 0 0] in 1
Got 17
rotating [0 0 1] in 1
rotating [0 0 1] in -1
rotating [ 0 -1  0] in -1
rotating [0 0 1] in -1
rotating [ 0 -1  0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in -1
rotating [ 0 -1  0] in -1
rotating [0 0 1] in -1
rotating [ 0 -1  0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in -1
rotating [0 0 1] in 1
rotating [0 0 1] in 1
rotating [0 0 1] in -1
rotating [1 0 0] in -1
rotating [0 0 1] in -1
rota

In [93]:
print(len(optimisedAns))
tot_steps+=len(optimisedAns)
steps_dir["2nd layer"]=len(optimisedAns)

30


In [94]:
printallfaces(face_matrix)

red
[[3 5 2]
 [1 1 1]
 [1 1 1]]
orange
[[4 1 5]
 [2 2 2]
 [2 2 2]]
green
[[5 5 2]
 [3 3 3]
 [3 3 3]]
blue
[[4 4 1]
 [4 4 4]
 [4 4 4]]
yellow
[[1 5 5]
 [5 5 3]
 [5 2 3]]
white
[[6 6 6]
 [6 6 6]
 [6 6 6]]


In [95]:
def thirdEdgeEle(face_matrix):
    yell_top=face_matrix[5,1]
    yell_right=face_matrix[5,2]
    yell_bottom=face_matrix[5,3]
    yell_left=face_matrix[5,4]
    
    face_dir=[]
    ele_pos=[]
    pattern=''
    front_face=np.array([1,0,0])
    index=[]
    
    face_dir.append(list(yell_top[1]))
    face_dir.append(list(yell_right[1]))
    face_dir.append(list(yell_bottom[1]))
    face_dir.append(list(yell_left[1]))
    
    ele_pos.append(yell_top[0])
    ele_pos.append(yell_right[0])
    ele_pos.append(yell_bottom[0])
    ele_pos.append(yell_left[0])
    
    
    if face_dir[0]!=[0,0,1] and face_dir[1]!=[0,0,1] and face_dir[2]!=[0,0,1] and face_dir[3]!=[0,0,1]:
        pattern='dot'
        return front_face,pattern
    elif face_dir[0]==[0,0,1] and face_dir[1]==[0,0,1] and face_dir[2]==[0,0,1] and face_dir[3]==[0,0,1]:
        pattern='cross'
        return front_face,pattern
    for i in range(4):
        if face_dir[i]==[0,0,1]:
            index.append(i)
    a=ele_pos[index[0]]-np.array([0,0,1])
    b=ele_pos[index[1]]-np.array([0,0,1])
    
    aCrossb=np.cross(a,b)
    
    if list(aCrossb)==[0,0,0]:
        pattern='bar'
        front_face=np.cross(np.array([0,0,1]),a)
        
        return front_face,pattern
    elif list(aCrossb)==[0,0,1]:
        pattern='L'
        front_face=-a
        return front_face,pattern
    elif list(aCrossb)==[0,0,-1]:
        pattern='L'
        front_face=-b
        return front_face,pattern
    
    

In [96]:
front_face,pattern=thirdEdgeEle(face_matrix)
print(front_face)
print(pattern)

[1 0 0]
L


In [97]:
def makeYellowCross(face_matrix):
    front_face,pattern=thirdEdgeEle(face_matrix)
    front_face=np.copy(front_face)
    up_face=np.array([0,0,1])
    right_face=np.cross(up_face,front_face)
    back_face=-front_face
    left_face=-right_face
    ans=[]
    
    if pattern=='bar':
        #F
        rotate(front_face,face_matrix,1)
        ans.append([front_face,1])
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #F'
        rotate(front_face,face_matrix,-1)
        ans.append([front_face,-1])
    elif pattern=='L':
        #F
        rotate(front_face,face_matrix,1)
        ans.append([front_face,1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #F'
        rotate(front_face,face_matrix,-1)
        ans.append([front_face,-1])
    elif pattern=='dot':
        #F
        rotate(front_face,face_matrix,1)
        ans.append([front_face,1])
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #S
        rotate(front_face,face_matrix,-1)
        rotate(back_face,face_matrix,1)
        ans.append([front_face,-1])
        ans.append([back_face,1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #L
        rotate(left_face,face_matrix,1)
        ans.append([left_face,1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #L'
        rotate(left_face,face_matrix,-1)
        ans.append([left_face,-1])
        #f'
        rotate(back_face,face_matrix,-1)
        ans.append([back_face,-1])
    return ans

In [98]:
ans=makeYellowCross(face_matrix)
print(len(ans))
tot_steps+=len(ans)
steps_dir["yellow cross"]=len(ans)

rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotating [0 1 0] in 1
rotating [0 0 1] in -1
rotating [0 1 0] in -1
rotating [1 0 0] in -1
6


In [99]:
printallfaces(face_matrix)

red
[[2 3 1]
 [1 1 1]
 [1 1 1]]
orange
[[4 2 4]
 [2 2 2]
 [2 2 2]]
green
[[3 1 5]
 [3 3 3]
 [3 3 3]]
blue
[[5 4 3]
 [4 4 4]
 [4 4 4]]
yellow
[[2 5 1]
 [5 5 5]
 [5 5 5]]
white
[[6 6 6]
 [6 6 6]
 [6 6 6]]


In [100]:
def thirdCornerPieces(face_matrix):
    top_right=face_matrix[5,5]
    bottom_right=face_matrix[5,6]
    bottom_left=face_matrix[5,7]
    top_left=face_matrix[5,8]
    ele_pos=[[-1,1,1],[1,1,1],[1,-1,1],[-1,-1,1]]
    
    face_dir=[1,2,3,4]
    
    face_dir[ele_pos.index(list(top_right[0]))]=top_right[1]
    face_dir[ele_pos.index(list(bottom_right[0]))]=bottom_right[1]
    face_dir[ele_pos.index(list(bottom_left[0]))]=bottom_left[1]
    face_dir[ele_pos.index(list(top_left[0]))]=top_left[1]
    
    front_face=np.array([0,0,0])
    pattern=''
    
    count=0
    face_index=[]
    
    for i in range(4):
        if list(face_dir[i])==[0,0,1]:
            count+=1
            face_index.append(i)
    
    if count==0:
        for i in range(4):
            if list(np.cross(face_dir[i],np.array([0,0,1])))==list(face_dir[(i+1)%4]) and list(face_dir[(i+3)%4])!=list(face_dir[i]):
                front_face=face_dir[i]
                pattern='Pi'
                return front_face,pattern
        front_face=face_dir[0]
        pattern='H'
        return front_face,pattern
    elif count==1:
        index=face_index[0]
        temp=[[-1,0,0],[0,1,0],[1,0,0],[0,-1,0]]
        if list(face_dir[(index+3)%4])==temp[index]:
            front_face=face_dir[(index+3)%4]
            pattern='sune'
            return front_face,pattern
        front_face=face_dir[(index+2)%4]
        pattern='antisune'
        return front_face,pattern
    elif count==2:
        index1=face_index[0]
        index2=face_index[1]
        temp=[[-1,0,0],[0,1,0],[1,0,0],[0,-1,0]]
        if index2-index1==2:
            pattern='L'
            if list(face_dir[(index1+1)%4])==temp[(index1+1)%4]:
                front_face=face_dir[(index1+1)%4]
                return front_face,pattern
            front_face=face_dir[(index2+1)%4]
            return front_face,pattern
        else:
            if list(face_dir[(index1+2)%4])==list(face_dir[(index2+2)%4]):
                pattern='U'
                front_face=face_dir[(index1+2)%4]
                return front_face,pattern
            pattern='T'
            if list(face_dir[(index1+2)%4])==temp[(index1+3)%4]:
                front_face=face_dir[(index1+2)%4]
                return front_face,pattern
            front_face=face_dir[(index2+2)%4]
            return front_face,pattern
    else:
        pattern='completed'
        return front_face,pattern
            

In [101]:
front_face,pattern=thirdCornerPieces(face_matrix)
print(pattern)
print(front_face)

T
[0 1 0]


In [102]:
def oll(face_matrix):
    front_face,pattern=thirdCornerPieces(face_matrix)
    front_face=np.copy(front_face)
    up_face=np.array([0,0,1])
    right_face=np.cross(up_face,front_face)
    left_face=-right_face
    down_face=-up_face
    back_face=-front_face
    
    ans=[]
    
    if pattern=='H':
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
    elif pattern=='Pi':
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
    elif pattern=='antisune':
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #U'
        rotate(up_face,face_matrix,-1)
        ans.append([up_face,-1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
    elif pattern=='sune':
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
    elif pattern=='L':
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #F
        rotate(front_face,face_matrix,1)
        ans.append([front_face,1])
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #B'
        rotate(back_face,face_matrix,-1)
        ans.append([back_face,-1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #F'
        rotate(front_face,face_matrix,-1)
        ans.append([front_face,-1])
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #B
        rotate(back_face,face_matrix,1)
        ans.append([back_face,1])
    elif pattern=='U':
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #D
        rotate(down_face,face_matrix,1)
        ans.append([down_face,1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #D'
        rotate(down_face,face_matrix,-1)
        ans.append([down_face,-1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #U
        rotate(up_face,face_matrix,1)
        ans.append([up_face,1])
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
    elif pattern == 'T':
        #R'
        rotate(right_face,face_matrix,-1)
        ans.append([right_face,-1])
        #F'
        rotate(front_face,face_matrix,-1)
        ans.append([front_face,-1])
        #L
        rotate(left_face,face_matrix,1)
        ans.append([left_face,1])
        #F
        rotate(front_face,face_matrix,1)
        ans.append([front_face,1])
        #R
        rotate(right_face,face_matrix,1)
        ans.append([right_face,1])
        #F'
        rotate(front_face,face_matrix,-1)
        ans.append([front_face,-1])
        #L'
        rotate(left_face,face_matrix,-1)
        ans.append([left_face,-1])
        #F
        rotate(front_face,face_matrix,1)
        ans.append([front_face,1])
    return ans

In [103]:
ans=oll(face_matrix)
print(len(ans))
tot_steps+=len(ans)
steps_dir["yellow top (OLL)"]=len(ans)

rotating [-1  0  0] in -1
rotating [0 1 0] in -1
rotating [1 0 0] in 1
rotating [0 1 0] in 1
rotating [-1  0  0] in 1
rotating [0 1 0] in -1
rotating [1 0 0] in -1
rotating [0 1 0] in 1
8


In [104]:
printallfaces(face_matrix)

red
[[1 3 1]
 [1 1 1]
 [1 1 1]]
orange
[[4 2 3]
 [2 2 2]
 [2 2 2]]
green
[[3 1 2]
 [3 3 3]
 [3 3 3]]
blue
[[2 4 4]
 [4 4 4]
 [4 4 4]]
yellow
[[5 5 5]
 [5 5 5]
 [5 5 5]]
white
[[6 6 6]
 [6 6 6]
 [6 6 6]]


# PLL Corner Pieces

In [105]:
face_dict={1:np.array([1,0,0]),2:np.array([-1,0,0]),3:np.array([0,1,0]),4:np.array([0,-1,0]),5:np.array([0,0,1]),6:np.array([0,0,-1])}

In [106]:
placed=[]
for i in range(1,5):
    if(np.array_equal(face_matrix[i][5][1],face_matrix[i][8][1])):
        placed.append(i)
steps=0        
if(len(placed)==0):
    #Diagonal Corner swap
    f=np.array([1,0,0])
    r=np.array([0,1,0])
    u=np.array([0,0,1])
    #fru'r'u'rur'f'rur'u'r'frf'
    rotate(f,face_matrix,1)
    rotate(r,face_matrix,1)
    rotate(u,face_matrix,-1)
    rotate(r,face_matrix,-1)
    rotate(u,face_matrix,-1)
    rotate(r,face_matrix,1)
    rotate(u,face_matrix,1)
    rotate(r,face_matrix,-1)
    rotate(f,face_matrix,-1)
    rotate(r,face_matrix,1)
    rotate(u,face_matrix,1)
    rotate(r,face_matrix,-1)
    rotate(u,face_matrix,-1)
    rotate(r,face_matrix,-1)
    rotate(f,face_matrix,1)
    rotate(r,face_matrix,1)
    rotate(f,face_matrix,-1)
    steps+=17
if(len(placed)==1):
    #Adjacent corner swap
    i=placed[0]
    #rur'u'r'frru'r'u'rur'f'
    u=np.array([0,0,1])
    f=np.bitwise_xor(face_matrix[i][5][0],face_matrix[i][5][1])
    f[2]=0
    r=-face_matrix[i][5][1]
    rotate(r,face_matrix,1)
    rotate(u,face_matrix,1)
    rotate(r,face_matrix,-1)
    rotate(u,face_matrix,-1)
    rotate(r,face_matrix,-1)
    rotate(f,face_matrix,1)
    rotate(r,face_matrix,1)
    rotate(r,face_matrix,1)
    rotate(u,face_matrix,-1)
    rotate(r,face_matrix,-1)
    rotate(u,face_matrix,-1)
    rotate(r,face_matrix,1)
    rotate(u,face_matrix,1)
    rotate(r,face_matrix,-1)
    rotate(f,face_matrix,-1)
    steps+=15

face=face_dict[1]    
if(np.array_equal(face_matrix[1][5][1],face )):
    pass
elif(np.array_equal(face_matrix[1][5][1],-face)):
    rotate(np.array([0,0,1]),face_matrix,1)
    rotate(np.array([0,0,1]),face_matrix,1)
    steps+=2
else:
    if((face_matrix[1][5][0][0]==face[0]) or(face_matrix[1][5][0][1]==face[1])):
        rotate(np.array([0,0,1]),face_matrix,-1)
    else:
        rotate(np.array([0,0,1]),face_matrix,1)
    steps+=1
print(steps) 
tot_steps+=steps
steps_dir["yellow Corner (PLL-1)"]=steps

rotating [-1  0  0] in 1
rotating [0 0 1] in 1
rotating [-1  0  0] in -1
rotating [0 0 1] in -1
rotating [-1  0  0] in -1
rotating [0 1 0] in 1
rotating [-1  0  0] in 1
rotating [-1  0  0] in 1
rotating [0 0 1] in -1
rotating [-1  0  0] in -1
rotating [0 0 1] in -1
rotating [-1  0  0] in 1
rotating [0 0 1] in 1
rotating [-1  0  0] in -1
rotating [0 1 0] in -1
15


In [107]:
printallfaces(face_matrix)

red
[[1 2 1]
 [1 1 1]
 [1 1 1]]
orange
[[2 3 2]
 [2 2 2]
 [2 2 2]]
green
[[3 1 3]
 [3 3 3]
 [3 3 3]]
blue
[[4 4 4]
 [4 4 4]
 [4 4 4]]
yellow
[[5 5 5]
 [5 5 5]
 [5 5 5]]
white
[[6 6 6]
 [6 6 6]
 [6 6 6]]


In [108]:
#below func. is used in end...start
def RotationForcorrectPos(face_matrix):
    if(list(face_matrix[1][1][1])!=list(np.array([1,0,0]))):
        vec=np.cross(face_matrix[1][1][1],np.array([1,0,0]))
        if(sum(vec)==0):
                     rotate(np.array([0,0,1]),face_matrix,1)
                     rotate(np.array([0,0,1]),face_matrix,1)
                     return 2
        elif(sum(vec)<0):
                     rotate(np.array([0,0,1]),face_matrix,1)
                     return 1
        elif(sum(vec)>0):
                     rotate(np.array([0,0,1]),face_matrix,-1)
                     return 1
#...end                   
dic={tuple(np.array([1,0,0])):1,tuple(np.array([-1,0,0])):2,tuple(np.array([0,1,0])):3,tuple(np.array([0,-1,0])):4}
correct_pos=[]
noOfOpp=0
steps=0
for i in range(1,5):
    if(list(face_dict[i])==list(face_matrix[i][1][1])):
        correct_pos.append(i)
    if(list(face_matrix[i][1][1].copy()*-1)==list(face_dict[i])):
        noOfOpp+=1
print(noOfOpp)
if(len(correct_pos)==1):
    right_face=np.cross(face_dict[correct_pos[0]],np.array([0,0,1]))
    left_face=right_face*-1
    print(right_face)
    print(left_face)
    print(dic[tuple(right_face)])
    print(dic[tuple(left_face)])
    print(face_matrix[dic[tuple(right_face)]][1][1])
    print(face_matrix[dic[tuple(left_face)]][1][1])
    if(list(face_matrix[dic[tuple(right_face)]][1][1])==list(left_face)):
        rotate(left_face,face_matrix,-1)
        rotate(np.array([0,0,1]),face_matrix,1)
        rotate(left_face,face_matrix,-1)
        rotate(np.array([0,0,1]),face_matrix,-1)
        rotate(left_face,face_matrix,-1)
        rotate(np.array([0,0,1]),face_matrix,-1)
        rotate(left_face,face_matrix,-1)
        rotate(np.array([0,0,1]),face_matrix,1)
        rotate(left_face,face_matrix,1)
        rotate(np.array([0,0,1]),face_matrix,1)
        rotate(left_face,face_matrix,1)
        rotate(left_face,face_matrix,1)
        steps+=12
    elif(list(face_matrix[dic[tuple(left_face)]][1][1])==list(right_face)):
        rotate(right_face,face_matrix,1)
        rotate(np.array([0,0,1]),face_matrix,-1)
        rotate(right_face,face_matrix,1)
        rotate(np.array([0,0,1]),face_matrix,1)
        rotate(right_face,face_matrix,1)
        rotate(np.array([0,0,1]),face_matrix,1)
        rotate(right_face,face_matrix,1)
        rotate(np.array([0,0,1]),face_matrix,-1)
        rotate(right_face,face_matrix,-1)
        rotate(np.array([0,0,1]),face_matrix,-1)
        rotate(right_face,face_matrix,1)
        rotate(right_face,face_matrix,1)
        steps+=12
        
if(len(correct_pos)==0 and noOfOpp==4):
    #m2 start
    rotate(np.array([0,-1,0]),face_matrix,1)
    rotate(np.array([0,-1,0]),face_matrix,1)
    rotate(np.array([0,1,0]),face_matrix,1)
    rotate(np.array([0,1,0]),face_matrix,1)   
    #m2 ends
    #u'
    rotate(np.array([0,0,-1]),face_matrix,-1)
    #m2 start
    rotate(np.array([0,-1,0]),face_matrix,1)
    rotate(np.array([0,-1,0]),face_matrix,1)
    rotate(np.array([0,1,0]),face_matrix,1)
    rotate(np.array([0,1,0]),face_matrix,1)   
    #m2 ends
    #u2 starts
    rotate(np.array([0,0,1]),face_matrix,1)
    rotate(np.array([0,0,1]),face_matrix,1)
    #u2 ends
    #m2 start
    rotate(np.array([0,-1,0]),face_matrix,1)
    rotate(np.array([0,-1,0]),face_matrix,1)
    rotate(np.array([0,1,0]),face_matrix,1)
    rotate(np.array([0,1,0]),face_matrix,1)   
    #m2 ends
    #u'
    rotate(np.array([0,0,-1]),face_matrix,-1)
    #m2 start
    rotate(np.array([0,-1,0]),face_matrix,1)
    rotate(np.array([0,-1,0]),face_matrix,1)
    rotate(np.array([0,1,0]),face_matrix,1)
    rotate(np.array([0,1,0]),face_matrix,1)   
    #m2 ends
    steps+=12
if(len(correct_pos)==0 and noOfOpp==0):
    right_face=np.array([0,-1,0])
    left_face=right_face*-1
    front_face=face_matrix[4][1][1].copy()
    if(sum(front_face)<0):
        print('1st one')
        #m' 
        rotate(right_face,face_matrix,-1)
        rotate(left_face,face_matrix,1)
        #u'
        rotate(front_face,face_matrix,-1)
        #m2 start
        rotate(np.array([0,-1,0]),face_matrix,1)
        rotate(np.array([0,-1,0]),face_matrix,1)
        rotate(np.array([0,1,0]),face_matrix,1)
        rotate(np.array([0,1,0]),face_matrix,1)
        #m2 end
        #u'
        rotate(front_face*-1,face_matrix,-1)
        #m2 start
        rotate(np.array([0,-1,0]),face_matrix,1)
        rotate(np.array([0,-1,0]),face_matrix,1)
        rotate(np.array([0,1,0]),face_matrix,1)
        rotate(np.array([0,1,0]),face_matrix,1)
        #m2 end
        #u'
        rotate(front_face,face_matrix,-1)
        #m' 
        rotate(right_face,face_matrix,-1)
        rotate(left_face,face_matrix,1)
        #u2 start
        rotate(np.array([0,0,-1]),face_matrix,1)
        rotate(np.array([0,0,-1]),face_matrix,1)
        #u2 ends
        #m2 start
        rotate(np.array([0,-1,0]),face_matrix,1)
        rotate(np.array([0,-1,0]),face_matrix,1)
        rotate(np.array([0,1,0]),face_matrix,1)
        rotate(np.array([0,1,0]),face_matrix,1)
        #m2 end
        steps+=13
        steps+=RotationForcorrectPos(face_matrix)
    elif(sum(front_face)>0):
        print("2nd one")
        #m' 
        rotate(left_face,face_matrix,-1)
        rotate(right_face,face_matrix,1)
        #u
        rotate(front_face,face_matrix,1)
        #m2 start
        rotate(np.array([0,-1,0]),face_matrix,1)
        rotate(np.array([0,-1,0]),face_matrix,1)
        rotate(np.array([0,1,0]),face_matrix,1)
        rotate(np.array([0,1,0]),face_matrix,1)
        #m2 end
        #u
        rotate(front_face*-1,face_matrix,1)
        #m2 start
        rotate(np.array([0,-1,0]),face_matrix,1)
        rotate(np.array([0,-1,0]),face_matrix,1)
        rotate(np.array([0,1,0]),face_matrix,1)
        rotate(np.array([0,1,0]),face_matrix,1)
        #m2 end
        #u
        rotate(front_face,face_matrix,1)
        #m' 
        rotate(left_face,face_matrix,-1)
        rotate(right_face,face_matrix,1)
        #u2 start
        rotate(np.array([0,0,-1]),face_matrix,1)
        rotate(np.array([0,0,-1]),face_matrix,1)
        #u2 ends
        #m2 start
        rotate(np.array([0,-1,0]),face_matrix,1)
        rotate(np.array([0,-1,0]),face_matrix,1)
        rotate(np.array([0,1,0]),face_matrix,1)
        rotate(np.array([0,1,0]),face_matrix,1)
        #m2 end
        steps+=13
        steps+=RotationForcorrectPos(face_matrix) 

1
[-1  0  0]
[1 0 0]
2
1
[1 0 0]
[0 1 0]
rotating [1 0 0] in -1
rotating [0 0 1] in 1
rotating [1 0 0] in -1
rotating [0 0 1] in -1
rotating [1 0 0] in -1
rotating [0 0 1] in -1
rotating [1 0 0] in -1
rotating [0 0 1] in 1
rotating [1 0 0] in 1
rotating [0 0 1] in 1
rotating [1 0 0] in 1
rotating [1 0 0] in 1


In [109]:
print("no. of steps " + str(steps))
printallfaces(face_matrix)
tot_steps+=steps
steps_dir["yellow Edge (PLL-2)"]=steps

no. of steps 12
red
[[1 1 1]
 [1 1 1]
 [1 1 1]]
orange
[[2 2 2]
 [2 2 2]
 [2 2 2]]
green
[[3 3 3]
 [3 3 3]
 [3 3 3]]
blue
[[4 4 4]
 [4 4 4]
 [4 4 4]]
yellow
[[5 5 5]
 [5 5 5]
 [5 5 5]]
white
[[6 6 6]
 [6 6 6]
 [6 6 6]]


In [110]:
for i in steps_dir:
    print(i,": ",steps_dir[i])

White cross :  12
White corner :  15
2nd layer :  30
yellow cross :  6
yellow top (OLL) :  8
yellow Corner (PLL-1) :  15
yellow Edge (PLL-2) :  12


In [111]:
print("total steps taken",tot_steps)

total steps taken 98
